# YOLOv7 Tiny IINT8 Quantization

### Imports

In [26]:
import os
import sys
# print(sys.path)
sys.path.append("/home/gjraza/aletheia_ai/interview_task/yolov7")
import torch 
print (torch.__version__)

import pytorch_quantization
print (pytorch_quantization.__version__)

1.8.0+cpu
2.1.0


In [27]:
import numpy as np
import torch
import yaml
from tqdm import tqdm
from pathlib import Path

from models.experimental import attempt_load
from utils.datasets import create_dataloader
from utils.general import coco80_to_coco91_class, check_dataset, check_file, check_img_size, check_requirements, \
    box_iou, non_max_suppression, scale_coords, xyxy2xywh, xywh2xyxy, set_logging, increment_path, colorstr
from utils.metrics import ap_per_class, ConfusionMatrix
from utils.plots import plot_images, output_to_target, plot_study_txt
from utils.torch_utils import select_device, time_synchronized, TracedModel


print ('imported')

imported


### FP32 model and dataset setup

In [38]:
single_cls = False
device = 'cpu'
weights  = '/home/gjraza/aletheia_ai/interview_task/yolov7/runs_colab/train/yolov7-tiny-crowd-human-1000-2/weights/best.pt'
imgsz = 640
single_cls = False
batch_size = 1
augment = False
save_hybrid = False 
conf_thres = 0.001
iou_thres = 0.5
verbose = True
training = False
save_dir = 'runs/test/yolov7-tiny-crowd-human-pt-quant-test'
data = '/home/gjraza/aletheia_ai/interview_task/yolov7/data/data_crowd_human_yolo.yaml'
task = 'val'


class options:
    def __init__(self):
        self.single_cls = False
        self.device = 'cpu'
        self.weights  = '/home/gjraza/aletheia_ai/interview_task/yolov7/runs_colab/train/yolov7-tiny-crowd-human-1000-2/weights/best.pt'
        self.imgsz = 640
        self.single_cls = False
        self.batch_size = 1
        self.augment = False
        self.save_hybrid = False 
        self.conf_thres = 0.001
        self.iou_thres = 0.5
        self.verbose = True
        self.training = False
        self.save_dir = 'runs/test/yolov7-tiny-crowd-human-pt-quant-test'
        self.data = '/home/gjraza/aletheia_ai/interview_task/yolov7/data/data_crowd_human_yolo.yaml'

opt = options()


In [39]:
opt.weights

'/home/gjraza/aletheia_ai/interview_task/yolov7/runs_colab/train/yolov7-tiny-crowd-human-1000-2/weights/best.pt'

In [40]:
model = attempt_load(opt.weights, map_location=opt.device)  # load FP32 model
model.eval()
# print (model)


I1203 21:11:27.792776 140684794753408 _utils.py:72] Input is fake quantized to 8 bits in QuantConv2d with axis None!
I1203 21:11:27.797751 140684794753408 _utils.py:75] Weight is fake quantized to 8 bits in QuantConv2d with axis 0!
I1203 21:11:27.801338 140684794753408 tensor_quantizer.py:101] Creating histogram calibrator
I1203 21:11:27.804241 140684794753408 tensor_quantizer.py:105] Creating Max calibrator
I1203 21:11:27.822010 140684794753408 _utils.py:72] Input is fake quantized to 8 bits in QuantConv2d with axis None!
I1203 21:11:27.834161 140684794753408 _utils.py:75] Weight is fake quantized to 8 bits in QuantConv2d with axis 0!
I1203 21:11:27.835983 140684794753408 tensor_quantizer.py:101] Creating histogram calibrator
I1203 21:11:27.837439 140684794753408 tensor_quantizer.py:105] Creating Max calibrator
I1203 21:11:27.857929 140684794753408 _utils.py:72] Input is fake quantized to 8 bits in QuantConv2d with axis None!
I1203 21:11:27.859054 140684794753408 _utils.py:75] Weight 

Fusing layers... 


I1203 21:11:27.987073 140684794753408 _utils.py:72] Input is fake quantized to 8 bits in QuantConv2d with axis None!
I1203 21:11:27.992374 140684794753408 _utils.py:75] Weight is fake quantized to 8 bits in QuantConv2d with axis 0!
I1203 21:11:27.997444 140684794753408 tensor_quantizer.py:101] Creating histogram calibrator
I1203 21:11:28.000707 140684794753408 tensor_quantizer.py:105] Creating Max calibrator
I1203 21:11:28.031716 140684794753408 _utils.py:72] Input is fake quantized to 8 bits in QuantConv2d with axis None!
I1203 21:11:28.035336 140684794753408 _utils.py:75] Weight is fake quantized to 8 bits in QuantConv2d with axis 0!
I1203 21:11:28.039470 140684794753408 tensor_quantizer.py:101] Creating histogram calibrator
I1203 21:11:28.048306 140684794753408 tensor_quantizer.py:105] Creating Max calibrator
I1203 21:11:28.074259 140684794753408 _utils.py:72] Input is fake quantized to 8 bits in QuantConv2d with axis None!
I1203 21:11:28.075912 140684794753408 _utils.py:75] Weight 

IDetect.fuse


Model(
  (model): Sequential(
    (0): Conv(
      (conv): QuantConv2d(
        3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
        (_input_quantizer): TensorQuantizer(8bit narrow fake per-tensor amax=dynamic calibrator=HistogramCalibrator scale=1.0 quant)
        (_weight_quantizer): TensorQuantizer(8bit narrow fake axis=0 amax=dynamic calibrator=MaxCalibrator scale=1.0 quant)
      )
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Conv(
      (conv): QuantConv2d(
        32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
        (_input_quantizer): TensorQuantizer(8bit narrow fake per-tensor amax=dynamic calibrator=HistogramCalibrator scale=1.0 quant)
        (_weight_quantizer): TensorQuantizer(8bit narrow fake axis=0 amax=dynamic calibrator=MaxCalibrator scale=1.0 quant)
      )
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Conv(
      (conv): QuantConv2d(
        64, 32, kernel_size=(1, 1), stride=(1, 1)
       

In [41]:
gs = max(int(model.stride.max()), 32)  # grid size (max stride)
imgsz = check_img_size(imgsz, s=gs)  # check img_size
print (imgsz)


640


### Vanilla FP32 Eval

In [32]:
def eval(model):

    
    single_cls = False
    device = 'cpu'
    imgsz = 640
    single_cls = False
    batch_size = 1
    augment = False
    save_hybrid = False 
    conf_thres = 0.001
    iou_thres = 0.5
    verbose = True
    training = False
    save_dir = 'runs/test/yolov7-tiny-crowd-human-pt-quant-test'
    gs = max(int(model.stride.max()), 32)  # grid size (max stride)
    imgsz = check_img_size(imgsz, s=gs)  # check img_size
    data = '/home/gjraza/aletheia_ai/interview_task/yolov7/data/data_crowd_human_yolo.yaml'



    if isinstance(opt.data, str):
        is_coco = opt.data.endswith('coco.yaml')
        with open(data) as f:
            data = yaml.load(f, Loader=yaml.SafeLoader)
    check_dataset(data)  # check
    nc = 1 if single_cls else int(data['nc'])  # number of classes
    iouv = torch.linspace(0.5, 0.95, 10).to(device)  # iou vector for mAP@0.5:0.95
    niou = iouv.numel()
    print (nc)
    print (iouv)
    print (niou)
    # Dataloader

    if device != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
    task = 'val'  # path to train/val/test images
    dataloader = create_dataloader(data[task], imgsz, batch_size, gs, opt, pad=0.5, rect=False,
                                    prefix=colorstr(f'{task}: '))[0]


    seen = 0
    confusion_matrix = ConfusionMatrix(nc=nc)
    names = {k: v for k, v in enumerate(model.names if hasattr(model, 'names') else model.module.names)}
    print ('names:\n', names)
    coco91class = coco80_to_coco91_class()
    s = ('%20s' + '%12s' * 6) % ('Class', 'Images', 'Labels', 'P', 'R', 'mAP@.5', 'mAP@.5:.95')
    p, r, f1, mp, mr, map50, map, t0, t1 = 0., 0., 0., 0., 0., 0., 0., 0., 0.
    loss = torch.zeros(3, device=device)
    jdict, stats, ap, ap_class, wandb_images = [], [], [], [], []


    for batch_i, (img, targets, paths, shapes) in enumerate(tqdm(dataloader, desc=s)):
            img = img.to(device, non_blocking=True)
            img =  img.float()  # uint8 to fp16/32
            img /= 255.0  # 0 - 255 to 0.0 - 1.0
            targets = targets.to(device)
            nb, _, height, width = img.shape  # batch size, channels, height, width
            # print (img.shape)
            with torch.no_grad():
                # Run model
                t = time_synchronized()
                out, train_out = model(img, augment=augment)  # inference and training outputs
                t0 += time_synchronized() - t

                # # Compute loss
                # if compute_loss:
                #     loss += compute_loss([x.float() for x in train_out], targets)[1][:3]  # box, obj, cls

                # Run NMS
                targets[:, 2:] *= torch.Tensor([width, height, width, height]).to(device)  # to pixels
                lb = [targets[targets[:, 0] == i, 1:] for i in range(nb)] if save_hybrid else []  # for autolabelling
                t = time_synchronized()
                out = non_max_suppression(out, conf_thres=conf_thres, iou_thres=iou_thres, labels=lb, multi_label=True)
                t1 += time_synchronized() - t

            # Statistics per image
            for si, pred in enumerate(out):
                labels = targets[targets[:, 0] == si, 1:]
                nl = len(labels)
                tcls = labels[:, 0].tolist() if nl else []  # target class
                path = Path(paths[si])
                seen += 1

                if len(pred) == 0:
                    if nl:
                        stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
                    continue

                # Predictions
                predn = pred.clone()
                scale_coords(img[si].shape[1:], predn[:, :4], shapes[si][0], shapes[si][1])  # native-space pred


                # Assign all predictions as incorrect
                correct = torch.zeros(pred.shape[0], niou, dtype=torch.bool, device=device)
                if nl:
                    detected = []  # target indices
                    tcls_tensor = labels[:, 0]

                    # target boxes
                    tbox = xywh2xyxy(labels[:, 1:5])
                    scale_coords(img[si].shape[1:], tbox, shapes[si][0], shapes[si][1])  # native-space labels

                    # Per target class
                    for cls in torch.unique(tcls_tensor):
                        ti = (cls == tcls_tensor).nonzero(as_tuple=False).view(-1)  # prediction indices
                        pi = (cls == pred[:, 5]).nonzero(as_tuple=False).view(-1)  # target indices

                        # Search for detections
                        if pi.shape[0]:
                            # Prediction to target ious
                            ious, i = box_iou(predn[pi, :4], tbox[ti]).max(1)  # best ious, indices

                            # Append detections
                            detected_set = set()
                            for j in (ious > iouv[0]).nonzero(as_tuple=False):
                                d = ti[i[j]]  # detected target
                                if d.item() not in detected_set:
                                    detected_set.add(d.item())
                                    detected.append(d)
                                    correct[pi[j]] = ious[j] > iouv  # iou_thres is 1xn
                                    if len(detected) == nl:  # all targets already located in image
                                        break

                # Append statistics (correct, conf, pcls, tcls)
                stats.append((correct.cpu(), pred[:, 4].cpu(), pred[:, 5].cpu(), tcls))


    # Compute statistics
    stats = [np.concatenate(x, 0) for x in zip(*stats)]  # to numpy
    if len(stats) and stats[0].any():
        p, r, ap, f1, ap_class = ap_per_class(*stats, plot=False, v5_metric=False, save_dir=save_dir, names=names)
        ap50, ap = ap[:, 0], ap.mean(1)  # AP@0.5, AP@0.5:0.95
        mp, mr, map50, map = p.mean(), r.mean(), ap50.mean(), ap.mean()
        nt = np.bincount(stats[3].astype(np.int64), minlength=nc)  # number of targets per class
    else:
        nt = torch.zeros(1)

    # Print results
    pf = '%20s' + '%12i' * 2 + '%12.3g' * 4  # print format
    print(pf % ('all', seen, nt.sum(), mp, mr, map50, map))

    # Print results per class
    if (verbose or (nc < 50 and not training)) and nc > 1 and len(stats):
        for i, c in enumerate(ap_class):
            print(pf % (names[c], seen, nt[c], p[i], r[i], ap50[i], ap[i]))

    # Print speeds
    t = tuple(x / seen * 1E3 for x in (t0, t1, t0 + t1)) + (imgsz, imgsz, batch_size)  # tuple
    if not training:
        print('Speed: %.1f/%.1f/%.1f ms inference/NMS/total per %gx%g image at batch-size %g' % t)


eval (model)

2
tensor([0.50000, 0.55000, 0.60000, 0.65000, 0.70000, 0.75000, 0.80000, 0.85000, 0.90000, 0.95000])
10


val: Scanning '/my_data/datasets/crowd_human_yolo_1000/valid/labels.cache' images and labels... 200 found, 0 missing, 0 empty, 0 corrupted: 100%|██████████| 200/200 [00:00<?, ?it/s]


names:
 {0: 'person', 1: 'head'}


               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100%|██████████| 200/200 [01:41<00:00,  1.97it/s]

                 all         200        8622       0.839       0.597       0.678       0.341
              person         200        4311       0.845       0.646       0.737        0.39
                head         200        4311       0.834       0.548        0.62       0.293
Speed: 460.7/4.2/464.9 ms inference/NMS/total per 640x640 image at batch-size 1


### Set up Pytorch TRT Quantization 

In [42]:
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import calib
from pytorch_quantization.tensor_quant import QuantDescriptor

from absl import logging
logging.set_verbosity(logging.INFO)  # Disable logging as they are too noisy in notebook

quant_desc_input = QuantDescriptor(calib_method='histogram')
quant_nn.QuantConv2d.set_default_quant_desc_input(quant_desc_input)
quant_nn.QuantLinear.set_default_quant_desc_input(quant_desc_input)

In [43]:
from pytorch_quantization import quant_modules
quant_modules.initialize()

In [44]:
# print (model)
out1, out2 = model(torch.zeros(1, 3, 352, 672).to(device).type_as(next(model.parameters())))

### Calibration Functions

In [45]:
def collect_stats(model, data_loader, num_batches):
    """Feed data to the network and collect statistic"""

    # Enable calibrators
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.disable_quant()
                module.enable_calib()
            else:
                module.disable()

    # for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
    #     model(image.cuda())
    #     if i >= num_batches:
    #         break
    for i, (img, _, _, _) in tqdm(enumerate(data_loader), total=num_batches):
        img = img.to(device, non_blocking=True)
        img =  img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        model(img, augment=augment)
        if i >= num_batches:
            break
    

    # Disable calibrators
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.enable_quant()
                module.disable_calib()
            else:
                module.enable()
            
def compute_amax(model, **kwargs):
    # Load calib result
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                if isinstance(module._calibrator, calib.MaxCalibrator):
                    module.load_calib_amax()
                else:
                    module.load_calib_amax(**kwargs)
#             print(F"{name:40}: {module}")
    # model.cuda()

#### Make Calibration Dataset and Dataloader

In [46]:
if isinstance(data, str):
    is_coco = data.endswith('coco.yaml')
    with open(data) as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)
check_dataset(data)  # check
print (data)
trt_calib_loader, trt_calib_dataset =  create_dataloader(data['train'], imgsz, batch_size, gs, opt, pad=0.5, rect=False,
                                prefix=colorstr('train: '))

len(trt_calib_dataset)

{'train': '/my_data/datasets/crowd_human_yolo_1000/train/images', 'val': '/my_data/datasets/crowd_human_yolo_1000/valid/images', 'nc': 2, 'names': ['person', 'head']}


train: Scanning '/my_data/datasets/crowd_human_yolo_1000/train/labels.cache' images and labels... 1000 found, 0 missing, 0 empty, 6 corrupted: 100%|██████████| 1000/1000 [00:00<?, ?it/s]


994

In [87]:
# for i in range (500): 
#     img, targets, paths, shapes  = next(iter(trt_calib_loader))
#     print (img.shape)
#     print (paths)

# img, targets, paths, shapes  = next(iter(trt_calib_loader))
# print (img)

In [47]:
num_batches = int(len(trt_calib_dataset)/batch_size)
print (num_batches)

with torch.no_grad():
    collect_stats(model, trt_calib_loader, num_batches=9)
    # compute_amax(detector.model, method="percentile", percentile=99.99)

I1203 21:12:08.212665 140684794753408 tensor_quantizer.py:183] Disable `quant` stage.
I1203 21:12:08.219638 140684794753408 tensor_quantizer.py:179] Enable HistogramCalibrator
I1203 21:12:08.221172 140684794753408 tensor_quantizer.py:183] Disable `quant` stage.
I1203 21:12:08.222434 140684794753408 tensor_quantizer.py:179] Enable MaxCalibrator
I1203 21:12:08.224323 140684794753408 tensor_quantizer.py:183] Disable `quant` stage.
I1203 21:12:08.227807 140684794753408 tensor_quantizer.py:179] Enable HistogramCalibrator
I1203 21:12:08.229250 140684794753408 tensor_quantizer.py:183] Disable `quant` stage.
I1203 21:12:08.235118 140684794753408 tensor_quantizer.py:179] Enable MaxCalibrator
I1203 21:12:08.236520 140684794753408 tensor_quantizer.py:183] Disable `quant` stage.
I1203 21:12:08.238334 140684794753408 tensor_quantizer.py:179] Enable HistogramCalibrator
I1203 21:12:08.239295 140684794753408 tensor_quantizer.py:183] Disable `quant` stage.
I1203 21:12:08.243273 140684794753408 tensor_q

994


I1203 21:12:08.415921 140684794753408 tensor_quantizer.py:183] Disable `quant` stage.
I1203 21:12:08.417652 140684794753408 tensor_quantizer.py:179] Enable HistogramCalibrator
I1203 21:12:08.418421 140684794753408 tensor_quantizer.py:183] Disable `quant` stage.
I1203 21:12:08.419198 140684794753408 tensor_quantizer.py:179] Enable MaxCalibrator
I1203 21:12:08.419956 140684794753408 tensor_quantizer.py:183] Disable `quant` stage.
I1203 21:12:08.420791 140684794753408 tensor_quantizer.py:179] Enable HistogramCalibrator
I1203 21:12:08.421563 140684794753408 tensor_quantizer.py:183] Disable `quant` stage.
I1203 21:12:08.422341 140684794753408 tensor_quantizer.py:179] Enable MaxCalibrator
I1203 21:12:08.428311 140684794753408 tensor_quantizer.py:183] Disable `quant` stage.
I1203 21:12:08.429331 140684794753408 tensor_quantizer.py:179] Enable HistogramCalibrator
I1203 21:12:08.430347 140684794753408 tensor_quantizer.py:183] Disable `quant` stage.
I1203 21:12:08.431214 140684794753408 tensor_q

In [ ]:
print (model)

In [48]:
with torch.no_grad():
    # for method in ["percentile", "mse", "entropy"]:
    for method in [ "mse","percentile", "entropy"]:
        percentile = 99.99
        print(F" ############  {method} calibration ######################")
        if(method == 'percentile'):
            compute_amax(model, method=method, percentile = percentile)
        else:
            compute_amax(model, method=method)
        
        
        eval(model)
        torch.save(model.state_dict(), f"tiny_yolo_v7_ch_{method}.pth")

W1203 21:13:38.451281 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1203 21:13:38.452245 140684794753408 tensor_quantizer.py:238] Call .cuda() if running on GPU after loading calibrated amax.
W1203 21:13:38.455153 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W1203 21:13:38.459455 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1203 21:13:38.465697 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([64, 1, 1, 1]).
W1203 21:13:38.467345 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1203 21:13:38.468469 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W1203 21:13:38.470915 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1203 21:13:38.471800 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=to

 ############  percentile calibration ######################


W1203 21:13:38.653696 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([128, 1, 1, 1]).
W1203 21:13:38.656292 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1203 21:13:38.657249 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([64, 1, 1, 1]).
W1203 21:13:38.659252 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1203 21:13:38.660947 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([64, 1, 1, 1]).
W1203 21:13:38.667715 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1203 21:13:38.670453 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([64, 1, 1, 1]).
W1203 21:13:38.671937 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1203 21:13:38.676986 140684794753408 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Si

In [ ]:
quant_nn.TensorQuantizer.use_fb_fake_quant = True
dummy_input = torch.randn(1, 3, 640, 640, device=device)


# enable_onnx_checker needs to be disabled. See notes below.
torch.onnx.export(
    model, dummy_input, "a.onnx", verbose=True, opset_version=13, enable_onnx_checker=False)

#### Compare speed of FP32 vs INT8 Model

In [1]:
import numpy as np
import onnxruntime
import time

In [5]:

def benchmark(model_path):
    session = onnxruntime.InferenceSession(model_path)
    input_name = session.get_inputs()[0].name

    total = 0.0
    runs = 10
    input_data = np.zeros((1, 3, 640, 640), np.float32)
    # Warming up
    _ = session.run([], {input_name: input_data})
    for i in range(runs):
        start = time.perf_counter()
        _ = session.run([], {input_name: input_data})
        end = (time.perf_counter() - start) * 1000
        total += end
        print(f"{end:.2f}ms")
    total /= runs
    print(f"Avg: {total:.2f}ms")


In [8]:
fp32_model_path = '/home/gjraza/aletheia_ai/interview_task/yolov7/runs_colab/train/yolov7-tiny-crowd-human-1000-2/weights/best.onnx'
int8_model_path =  "yolov7-tiny-ch-int8.onnx"
print("benchmarking fp32 model...")
benchmark(fp32_model_path)

print("benchmarking int8 model...")
benchmark(int8_model_path)

benchmarking fp32 model...
192.95ms
187.73ms
192.39ms
188.09ms
162.90ms
149.94ms
151.48ms
157.76ms
172.81ms
175.87ms
Avg: 173.19ms
benchmarking int8 model...
299.21ms
284.16ms
220.35ms
182.48ms
316.36ms
339.97ms
202.65ms
207.60ms
218.38ms
234.06ms
Avg: 250.52ms


#### Checking Quantized ONNX Accuracy

In [17]:
session = onnxruntime.InferenceSession(int8_model_path)
input_name = session.get_inputs()[0].name

output_names  = session.get_outputs()
# for name in output_names:
#     print(name.name)
#     print(name.shape)

# print (input_name)

# outs = session.run([], {input_name: input_data})
# print (outs[0].shape)

In [25]:
if isinstance(data, str):
    is_coco = data.endswith('coco.yaml')
    with open(data) as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)
check_dataset(data)  # check
nc = 1 if single_cls else int(data['nc'])  # number of classes
iouv = torch.linspace(0.5, 0.95, 10).to(device)  # iou vector for mAP@0.5:0.95
niou = iouv.numel()
print (nc)
print (iouv)
print (niou)
# Dataloader

if device != 'cpu':
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
task = 'val'  # path to train/val/test images
dataloader = create_dataloader(data[task], imgsz, batch_size, gs, opt, pad=0.5, rect=False,
                                prefix=colorstr(f'{task}: '))[0]


seen = 0
confusion_matrix = ConfusionMatrix(nc=nc)
names = {k: v for k, v in enumerate(model.names if hasattr(model, 'names') else model.module.names)}
print ('names:\n', names)
coco91class = coco80_to_coco91_class()
s = ('%20s' + '%12s' * 6) % ('Class', 'Images', 'Labels', 'P', 'R', 'mAP@.5', 'mAP@.5:.95')
p, r, f1, mp, mr, map50, map, t0, t1 = 0., 0., 0., 0., 0., 0., 0., 0., 0.
loss = torch.zeros(3, device=device)
jdict, stats, ap, ap_class, wandb_images = [], [], [], [], []


for batch_i, (img, targets, paths, shapes) in enumerate(tqdm(dataloader, desc=s)):
        img = img.to(device, non_blocking=True)
        img =  img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        targets = targets.to(device)
        nb, _, height, width = img.shape  # batch size, channels, height, width
        # print (img.shape)
        with torch.no_grad():
            # Run model
            t = time_synchronized()
            # out, train_out = model(img, augment=augment)  # inference and training outputs
            out = session.run([], {input_name: img.numpy()})
            out = torch.from_numpy(out[0]) 
            t0 += time_synchronized() - t

            # # Compute loss
            # if compute_loss:
            #     loss += compute_loss([x.float() for x in train_out], targets)[1][:3]  # box, obj, cls

            # Run NMS
            targets[:, 2:] *= torch.Tensor([width, height, width, height]).to(device)  # to pixels
            lb = [targets[targets[:, 0] == i, 1:] for i in range(nb)] if save_hybrid else []  # for autolabelling
            t = time_synchronized()
            out = non_max_suppression(out, conf_thres=conf_thres, iou_thres=iou_thres, labels=lb, multi_label=True)
            t1 += time_synchronized() - t

        # Statistics per image
        for si, pred in enumerate(out):
            labels = targets[targets[:, 0] == si, 1:]
            nl = len(labels)
            tcls = labels[:, 0].tolist() if nl else []  # target class
            path = Path(paths[si])
            seen += 1

            if len(pred) == 0:
                if nl:
                    stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
                continue

            # Predictions
            predn = pred.clone()
            scale_coords(img[si].shape[1:], predn[:, :4], shapes[si][0], shapes[si][1])  # native-space pred


            # Assign all predictions as incorrect
            correct = torch.zeros(pred.shape[0], niou, dtype=torch.bool, device=device)
            if nl:
                detected = []  # target indices
                tcls_tensor = labels[:, 0]

                # target boxes
                tbox = xywh2xyxy(labels[:, 1:5])
                scale_coords(img[si].shape[1:], tbox, shapes[si][0], shapes[si][1])  # native-space labels

                # Per target class
                for cls in torch.unique(tcls_tensor):
                    ti = (cls == tcls_tensor).nonzero(as_tuple=False).view(-1)  # prediction indices
                    pi = (cls == pred[:, 5]).nonzero(as_tuple=False).view(-1)  # target indices

                    # Search for detections
                    if pi.shape[0]:
                        # Prediction to target ious
                        ious, i = box_iou(predn[pi, :4], tbox[ti]).max(1)  # best ious, indices

                        # Append detections
                        detected_set = set()
                        for j in (ious > iouv[0]).nonzero(as_tuple=False):
                            d = ti[i[j]]  # detected target
                            if d.item() not in detected_set:
                                detected_set.add(d.item())
                                detected.append(d)
                                correct[pi[j]] = ious[j] > iouv  # iou_thres is 1xn
                                if len(detected) == nl:  # all targets already located in image
                                    break

            # Append statistics (correct, conf, pcls, tcls)
            stats.append((correct.cpu(), pred[:, 4].cpu(), pred[:, 5].cpu(), tcls))


# Compute statistics
stats = [np.concatenate(x, 0) for x in zip(*stats)]  # to numpy
if len(stats) and stats[0].any():
    p, r, ap, f1, ap_class = ap_per_class(*stats, plot=False, v5_metric=False, save_dir=save_dir, names=names)
    ap50, ap = ap[:, 0], ap.mean(1)  # AP@0.5, AP@0.5:0.95
    mp, mr, map50, map = p.mean(), r.mean(), ap50.mean(), ap.mean()
    nt = np.bincount(stats[3].astype(np.int64), minlength=nc)  # number of targets per class
else:
    nt = torch.zeros(1)

# Print results
pf = '%20s' + '%12i' * 2 + '%12.3g' * 4  # print format
print(pf % ('all', seen, nt.sum(), mp, mr, map50, map))

# Print results per class
if (verbose or (nc < 50 and not training)) and nc > 1 and len(stats):
    for i, c in enumerate(ap_class):
        print(pf % (names[c], seen, nt[c], p[i], r[i], ap50[i], ap[i]))

# Print speeds
t = tuple(x / seen * 1E3 for x in (t0, t1, t0 + t1)) + (imgsz, imgsz, batch_size)  # tuple
if not training:
    print('Speed: %.1f/%.1f/%.1f ms inference/NMS/total per %gx%g image at batch-size %g' % t)

2
tensor([0.50000, 0.55000, 0.60000, 0.65000, 0.70000, 0.75000, 0.80000, 0.85000, 0.90000, 0.95000])
10


val: Scanning '/my_data/datasets/crowd_human_yolo_1000/valid/labels.cache' images and labels... 200 found, 0 missing, 0 empty, 0 corrupted: 100%|██████████| 200/200 [00:00<?, ?it/s]


names:
 {0: 'person', 1: 'head'}


               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100%|██████████| 200/200 [00:50<00:00,  3.99it/s]

                 all         200        8622         0.8       0.613       0.675       0.341
              person         200        4311       0.809       0.661       0.735        0.39
                head         200        4311       0.792       0.564       0.615       0.292
Speed: 203.4/4.1/207.4 ms inference/NMS/total per 640x640 image at batch-size 1
